## Notebook for deploying Cognite Function

### Authentication

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import os

from initialize import initialize_client

func_suffix = "Development"

In [2]:
ts_input_name = "VAL_11-LT-95034A:X.Value"
ts_output_name = "VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue"#f"VAL_11-LT-95034A:X.D.LeakValue"
tank_volume = 1400
derivative_value_excl = 0.002
# start_date = datetime(2023, 3, 21, 1, 0, 0)
function_name = "avg-drainage-rate-new"
schedule_name = "avg-drainage-rate-schedule-new"

data_dict = {'tot_days':0, 'tot_minutes':15, # convert date to str to make it JSON serializable
            'ts_input_name':ts_input_name, 'ts_output_name':ts_output_name,
            'derivative_value_excl':derivative_value_excl, 'tank_volume':tank_volume,
            'cdf_env':"dev", 'dataset_id': 1832663593546318,
            'backfill': False, 'backfill_period': 7,
            'function_name': function_name, 'schedule_name': schedule_name} # NB: change dataset id when going to dev/test/prod!

cron_interval = "3" # 15

In [3]:
client = initialize_client(data_dict["cdf_env"], cache_token=False)

### Upload file to be associated with your dataset

In [10]:
folder = os.getcwd().replace("\\", "/")
# function_path = "handler.py"
function_path = "zippy.zip"

uploaded = client.files.upload(path=f"{folder}/{function_path}", name=function_path, data_set_id=data_dict["dataset_id"])

### Deploy Cognite function

In [11]:
# Create function
func_drainage = client.functions.create(
    name=f"avg-drainage-rate-new",
    external_id=f"avg-drainage-rate-new",
    # folder=".",
    file_id=uploaded.id,
)

In [12]:
func_drainage = client.functions.retrieve(external_id=f"avg-drainage-rate-new")
func_drainage.status

'Deploying'

In [68]:
# Single call to function
call_func_drainage = func_drainage.call(data=data_dict)

### Schedule Cognite function

In [13]:
# Schedule
func_drainage_schedule = client.functions.schedules.create(
    name=f"avg-drainage-rate-schedule-new",
    cron_expression=f"*/{cron_interval} * * * *", # every 15 min
    function_id=func_drainage.id,
    #client_credentials=client,
    description="Calculation scheduled every hour",
    data=data_dict
)

### NB: Delete function
Hear with DataOps: how do you know when there is a backfilling in CDF for PI time series - are there automatic checks for this, what is best practice?

In [19]:
client.functions.delete(id=func_drainage.id)

client.time_series.delete(external_id=f"VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue{func_suffix}")

## Experimental